In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

: 

In [2]:
import pandas as pd
path = 'C:/Users/withus/Desktop/sabin/Trend-analysis/Keyphrase_classification/dataset/'
df = pd.read_csv(path+'기술과학분야_train_set.csv',index_col=0)
df_val = pd.read_csv(path+'기술과학분야_valid_set.csv',index_col=0)

In [3]:
df = df.drop(index=df[df['domain'] == '기후'].index)
df = df.reset_index(drop=True)

In [4]:

train = pd.concat([df.loc[df['domain']=='기술과학'].sample(150000),df.loc[df['domain']=='세계'].sample(150000),df.loc[df['domain']=='정치'].sample(150000),df.loc[df['domain']=='경제'].sample(150000)]).sample(600000).reset_index(drop=True)
valid = pd.concat([df.loc[df['domain']=='기술과학'].sample(5000),df.loc[df['domain']=='세계'].sample(5000),df.loc[df['domain']=='정치'].sample(5000),df.loc[df['domain']=='경제'].sample(5000)]).sample(20000).reset_index(drop=True)

train['labels'] = train['domain'].replace(['세계', '경제', '기술과학','정치'],[0, 1, 2, 3])
valid['labels'] = valid['domain'].replace(['세계', '경제', '기술과학','정치'],[0, 1, 2, 3])

# 모델링

In [ ]:
! pip install transformers

: 

In [18]:
from tqdm import tqdm

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from transformers import AutoModel, AutoTokenizer,BertForSequenceClassification


model = BertForSequenceClassification.from_pretrained("klue/bert-base",num_labels=4)
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")
# 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [6]:
class CustomDataset(Dataset):

    def __init__(self,text, label,tokenizer, max_len):
        self.tokenizer = tokenizer
        self.text = text
        self.labels = label
        self.max_len = max_len

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        text = str(self.text[index])
        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'input_ids': torch.tensor(ids, dtype=torch.long),
            'attention_mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'labels': torch.tensor(self.labels[index], dtype=torch.float),
            }
    


training_set = CustomDataset(train['ko'],train['labels'], tokenizer, 128)
testing_set = CustomDataset(valid['ko'], valid['labels'],tokenizer, 128)

train_params = {'batch_size': 32,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': 32,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [7]:
loss_fn = nn.CrossEntropyLoss() 

LEARNING_RATE = 5e-5
optimizer = torch.optim.AdamW(params =  model.parameters(), lr=LEARNING_RATE)


train_steps = len(training_loader.dataset) // train_params['batch_size']
val_steps = len(testing_loader.dataset) // test_params['batch_size']


# 모델 저장용 PATH 따로 설정해죠!! 존재하는 디렉토리로 설정해줘야됑

In [ ]:
from tqdm import tqdm
epochs= 20
PATH = "따로 설정해줘!!!"

for epoch in (range(epochs)):
    model.train()
    num_correct = 0
    trainning_loss = 0
    for i, data in enumerate(tqdm(training_loader)):
        
        ids = data['input_ids'].to(device, dtype = torch.long)
        mask = data['attention_mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        label = data['labels'].to(device, dtype = torch.long)

        optimizer.zero_grad()
        
        output = model(ids, mask, token_type_ids)
        print(output)
        loss = loss_fn(output[0], label)
        trainning_loss += loss
        loss.backward()
        optimizer.step()
        if i % 1000 == 0:
          print("Training Losses: {}".format(trainning_loss/i+1))
          print("Trainong Accuracy:{}".format(num_correct/i+1))
    
        pred = output[0].argmax(dim=1)
        num_correct += torch.eq(pred,label).sum().float().item()
        


    print('EPOCH ', epoch+1)
    print("Training Losses: {}".format(trainning_loss/len(training_loader.dataset)))
    print("Trainong Accuracy:{}".format(num_correct/len(training_loader.dataset)))
    
    
    
    if epoch % 5 == 0 or epoch == epochs-1:
        torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'loss': trainning_loss,
                    }, PATH+f"model600000data{epoch}.pt")
  

: 

In [24]:
checkpoint = torch.load('C:/Users/withus/Desktop/sabin/Trend-analysis/Keyphrase_classification/model_output/model600000data10.pt')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

In [25]:
model.eval()
num_correct = 0

for i, data in enumerate(tqdm(testing_loader)):

      ids = data['input_ids'].to(device, dtype = torch.long)
      mask = data['attention_mask'].to(device, dtype = torch.long)
      token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
      label = data['labels'].to(device, dtype = torch.long)


      optimizer.zero_grad()

      output = model(ids, mask, token_type_ids)
      loss = loss_fn(output[0], label)
      optimizer.step()

      pred = output[0].argmax(dim=1)
      num_correct += torch.eq(pred, label).sum().float().item()


print('EPOCH ', epoch+1)
print("validation Losses: {}".format(loss))
print("validation Accuracy:{}".format(num_correct/len(testing_loader.dataset)))


100%|██████████| 625/625 [00:32<00:00, 19.02it/s]

EPOCH  11
validation Losses: 1.3947672843933105
validation Accuracy:0.25


# keyphrase classification(키워드 뽑은거 돌려 보는 부분 부분)


In [ ]:
import json

file_path = "/content/drive/MyDrive/data/keywords.json"

with open(file_path, 'r') as file:
    keywords = json.load(file)
    

: 

In [ ]:
category = ['사회', '경제', '기술과학','정치']
keyword_params = {'batch_size': 5,
                'shuffle': True,
                'num_workers': 0
                }
for keyword in keywords:
  keyword_5 = list(keyword.values())[0]
  keyword_set = CustomDataset(keyword_5,[-1]*len(keyword_5), tokenizer, 128)
  keyword_loader = DataLoader(keyword_set, **keyword_params)
  for i, data in enumerate((keyword_loader)):

    ids = data['input_ids'].to(device, dtype = torch.long)
    mask = data['attention_mask'].to(device, dtype = torch.long)
    token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
    label = data['labels'].to(device, dtype = torch.long)

    output = model(ids, mask, token_type_ids)
    
    pred = pd.Series(output[0].argmax(dim=1).cpu().numpy()).replace([0,1,2,3],category)
    print([(keyword_5[i],pred[i]) for i in range(5)])

: 